This notebook cleans structured data and appends it to cxr features.

# Environment

In [2]:
!pip install google-colab -q
!pip install shap -q
!pip install seaborn
!pip install tableone -q
!pip install sqldf

     |████████████████████████████████| 358kB 4.0MB/s 


In [3]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  84566      0 --:--:-- --:--:-- --:--:-- 84566
OK
65 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 65 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [4]:
#Standard library imports
from google.colab import auth
auth.authenticate_user()

In [5]:
!mkdir data
!gcsfuse hoboken_structured_data data

!mkdir features
!gcsfuse hoboken_cxr_features features

!mkdir final_validation_datasets
!gcsfuse final_validation_datasets final_validation_datasets

2021/05/17 20:17:36.316502 Using mount point: /content/data
2021/05/17 20:17:36.324421 Opening GCS connection...
2021/05/17 20:17:36.495434 Mounting file system "hoboken_structured_data"...
2021/05/17 20:17:36.527538 File system has been successfully mounted.
2021/05/17 20:17:36.759766 Using mount point: /content/features
2021/05/17 20:17:36.766863 Opening GCS connection...
2021/05/17 20:17:37.018734 Mounting file system "hoboken_cxr_features"...
2021/05/17 20:17:37.019103 File system has been successfully mounted.
2021/05/17 20:17:37.188249 Using mount point: /content/final_validation_datasets
2021/05/17 20:17:37.197428 Opening GCS connection...
2021/05/17 20:17:37.379628 Mounting file system "final_validation_datasets"...
2021/05/17 20:17:37.380018 File system has been successfully mounted.


# Libraries

In [6]:
import os, sys, math
from tensorflow.python.lib.io import file_io
import glob
import warnings


#Third party library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tableone import TableOne
from scipy.stats import uniform, randint
from scipy.stats.mstats import winsorize
import seaborn as sns
import shap
import sqldf as sql
import math
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, precision_recall_curve, plot_precision_recall_curve, average_precision_score, brier_score_loss, roc_curve
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_sample_weight

#Global configuration
pd.options.mode.chained_assignment = None
seed = 2020
np.random.seed(seed)

pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


# Tables ingestion

In [7]:
# the only files that contain the variables we are interested in are the following
hoboken_structured_data = pd.read_excel('data/hoboken_structured_data.xlsx', sheet_name='Sheet1')
hoboken_image_names = pd.read_csv('features/test-hoboken_image_names.csv', header=None)
#extracting patient id with cxr from path
hoboken_image_names = hoboken_image_names[0].str.extract('(\d+)').astype('int32')
hoboken_image_names=pd.to_numeric(hoboken_image_names[0])

invalid escape sequence \d
invalid escape sequence \d
invalid escape sequence \d


# Data Wrangling

# Exclusion criteria

In [8]:
print(':::Exclusion criteria:::')
print(f'Initial number of cases (hoboken_structured_data): {len(hoboken_structured_data)}')

hoboken_structured_data = hoboken_structured_data[hoboken_structured_data['age'] >= 16]
print(f'After excluding those with <16 Age: {len(hoboken_structured_data)}')



hoboken_structured_data =  hoboken_structured_data[hoboken_structured_data.index.isin(hoboken_image_names)]
print(f'After excluding those missing cxr: {len(hoboken_structured_data)}')

# we don't have admission_datetime in snuh_combined
#snuh_combined = snuh_combined[snuh_combined["admission_datetime"].isnull() != True]
#print(f'After excluding those with missing admission time: {len(snuh_combined)}')

# hospital_outcome is empty when =0 in this dataset, so removing empty ones would remove survivors. 
#hoboken_structured_data = hoboken_structured_data[hoboken_structured_data['hospital_outcome'].isnull()==False]
#print(f'After excluding those with missing hospital_outcome: {len(hoboken_structured_data)}')

# we are replacing missing cells with 0
hoboken_structured_data.hospital_outcome=hoboken_structured_data.hospital_outcome.fillna(0)

# Outcome distribution
print(':::Outcome distribution:::')
## inspect outcome distribution
print('Breakdown of hospital_outcome:')
print(hoboken_structured_data['hospital_outcome'].value_counts())


:::Exclusion criteria:::
Initial number of cases (hoboken_structured_data): 242
After excluding those with <16 Age: 242
After excluding those missing cxr: 200
:::Outcome distribution:::
Breakdown of hospital_outcome:
0.0    117
1.0     83
Name: hospital_outcome, dtype: int64


# Data Cleansing

In [9]:
#Vital signs
features_list = [name for name in list(hoboken_structured_data) if 'vitals' in name]

#Apply some clinical heuristics for valid ranges
limits = {'_sbp_':[20,240],
          '_hr_':[20,300],
          '_spo2_':[1,100],
          '_temp_':[30,45]}
for substr in limits.keys():
    for name in list(hoboken_structured_data):
        if substr in name:
            hoboken_structured_data[name][hoboken_structured_data[name] < limits[substr][0]] = np.nan
            hoboken_structured_data[name][hoboken_structured_data[name] > limits[substr][1]] = np.nan 

keep = [name for name in hoboken_structured_data.columns if 'last' not in name]
hoboken_structured_data = hoboken_structured_data.loc[:,keep]

# encoding sex the same way it is encoded in the reference dataset (HM COVIDDSL dataset)
print(hoboken_structured_data.sex.value_counts())

dictionary = {0: "MALE", 1: "FEMALE"}
hoboken_structured_data = hoboken_structured_data.replace({"sex": dictionary})

hoboken_structured_data.sex.value_counts()

hoboken_structured_data.head()

0    116
1     84
Name: sex, dtype: int64


,patient ID,age,sex,admission_datetime,ed_diagnosis,mechvent_flag,vitals_temp_ed_first,vitals_sbp_ed_first,vitals_dbp_ed_first,vitals_hr_ed_first,vitals_spo2_ed_first,hospital_outcome,pmhx_diabetes,pmhx_hld,pmhx_htn,pmhx_ihd,pmhx_ckd,pmhx_copd,pmhx_asthma,pmhx_activecancer,pmhx_chronicliver,pmhx_stroke,pmhx_chf,pmhx_dementia,lab_sodium,lab_leukocyte,lab_mean_platelet_volume,lab_neutrophil,lab_ddimer,lab_inr,lab_mch,lab_creatinine,lab_mcv,lab_aptt,lab_platelet,lab_lymphocyte_percentage,lab_glucose,lab_neutrophil_percentage,lab_ldh,lab_prothrombin_activity,lab_urea,lab_lymphocyte,lab_crp,lab_rdw,lab_hemoglobin,lab_rbc,lab_hct,lab_potassium,lab_alt,lab_ast
1,2,97,MALE,2020-04-05,NaN,0,38.444444,99,65,118,100.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,150.0,5.7,8.0,4.2,NaN,1.5,28.2,1.8,87.2,40.7,142.0,14.035088,110.0,73.684211,3059.0,17.7,37.0,0.8,44.0,15.4,16.2,5.72,49.9,4.1,104.0,83.0
2,3,41,MALE,2020-04-04,NaN,0,37.388889,130,83,91,94.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,5.0,7.7,2.4,200,1.1,33.2,0.9,94.4,32.8,191.0,42.000000,112.0,48.000000,714.0,12.7,14.0,2.1,24.1,12.8,17.9,5.38,50.8,3.7,110.0,83.0
3,4,71,FEMALE,2020-04-04,NaN,0,37.000000,143,77,89,99.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.0,4.1,8.2,2.9,209,1.2,28.6,0.7,84.6,31.8,217.0,19.512195,103.0,70.731707,528.0,14.3,20.0,0.8,17.9,13.6,13.3,4.66,39.4,3.8,28.0,39.0
4,5,72,FEMALE,2020-04-03,NaN,0,36.666667,154,99,74,94.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135.0,4.8,7.2,2.6,NaN,1.1,29.2,0.5,86.4,37.0,202.0,22.916667,125.0,54.166667,688.0,12.6,8.0,1.1,85.1,12.9,14.0,4.80,41.4,4.0,42.0,56.0
8,9,69,FEMALE,2020-04-03,NaN,0,37.333333,115,71,97,95.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0,7.8,9.2,6.2,393,1.1,29.6,0.5,89.5,32.1,295.0,12.820513,116.0,79.487179,1183.0,12.8,9.0,1.0,185.6,13.7,12.6,4.26,38.1,3.6,53.0,71.0


# Initial data exploration

In [10]:
hoboken_structured_data.columns

Index(['patient ID', 'age', 'sex', 'admission_datetime', 'ed_diagnosis',
       'mechvent_flag', 'vitals_temp_ed_first', 'vitals_sbp_ed_first',
       'vitals_dbp_ed_first', 'vitals_hr_ed_first', 'vitals_spo2_ed_first',
       'hospital_outcome', 'pmhx_diabetes', 'pmhx_hld', 'pmhx_htn', 'pmhx_ihd',
       'pmhx_ckd', 'pmhx_copd', 'pmhx_asthma', 'pmhx_activecancer',
       'pmhx_chronicliver', 'pmhx_stroke', 'pmhx_chf', 'pmhx_dementia',
       'lab_sodium', 'lab_leukocyte', 'lab_mean_platelet_volume',
       'lab_neutrophil', 'lab_ddimer', 'lab_inr', 'lab_mch', 'lab_creatinine',
       'lab_mcv', 'lab_aptt', 'lab_platelet', 'lab_lymphocyte_percentage',
       'lab_glucose', 'lab_neutrophil_percentage', 'lab_ldh',
       'lab_prothrombin_activity', 'lab_urea', 'lab_lymphocyte', 'lab_crp',
       'lab_rdw', 'lab_hemoglobin', 'lab_rbc', 'lab_hct', 'lab_potassium',
       'lab_alt', 'lab_ast'],
      dtype='object')

In [11]:
#remove labs that have >50% missing values

remove_lab = hoboken_structured_data.loc[:, hoboken_structured_data.columns.str.startswith('lab_')].isnull().sum().index[(hoboken_structured_data.loc[:, hoboken_structured_data.columns.str.startswith('lab_')].isnull().sum()/len(hoboken_structured_data.loc[:, hoboken_structured_data.columns.str.startswith('lab_')]) > 0.50)]
hoboken_structured_data=hoboken_structured_data.drop(columns=remove_lab)
hoboken_structured_data.loc[:, hoboken_structured_data.columns.str.startswith('lab_')].head()

,lab_sodium,lab_leukocyte,lab_mean_platelet_volume,lab_neutrophil,lab_inr,lab_mch,lab_creatinine,lab_mcv,lab_aptt,lab_platelet,lab_lymphocyte_percentage,lab_glucose,lab_neutrophil_percentage,lab_ldh,lab_prothrombin_activity,lab_urea,lab_lymphocyte,lab_crp,lab_rdw,lab_hemoglobin,lab_rbc,lab_hct,lab_potassium,lab_alt,lab_ast
1,150.0,5.7,8.0,4.2,1.5,28.2,1.8,87.2,40.7,142.0,14.035088,110.0,73.684211,3059.0,17.7,37.0,0.8,44.0,15.4,16.2,5.72,49.9,4.1,104.0,83.0
2,137.0,5.0,7.7,2.4,1.1,33.2,0.9,94.4,32.8,191.0,42.000000,112.0,48.000000,714.0,12.7,14.0,2.1,24.1,12.8,17.9,5.38,50.8,3.7,110.0,83.0
3,137.0,4.1,8.2,2.9,1.2,28.6,0.7,84.6,31.8,217.0,19.512195,103.0,70.731707,528.0,14.3,20.0,0.8,17.9,13.6,13.3,4.66,39.4,3.8,28.0,39.0
4,135.0,4.8,7.2,2.6,1.1,29.2,0.5,86.4,37.0,202.0,22.916667,125.0,54.166667,688.0,12.6,8.0,1.1,85.1,12.9,14.0,4.80,41.4,4.0,42.0,56.0
8,136.0,7.8,9.2,6.2,1.1,29.6,0.5,89.5,32.1,295.0,12.820513,116.0,79.487179,1183.0,12.8,9.0,1.0,185.6,13.7,12.6,4.26,38.1,3.6,53.0,71.0


In [12]:
hoboken_structured_data.describe()

,patient ID,age,ed_diagnosis,mechvent_flag,vitals_temp_ed_first,vitals_sbp_ed_first,vitals_dbp_ed_first,vitals_hr_ed_first,vitals_spo2_ed_first,hospital_outcome,pmhx_diabetes,pmhx_hld,pmhx_htn,pmhx_ihd,pmhx_ckd,pmhx_copd,pmhx_asthma,pmhx_activecancer,pmhx_chronicliver,pmhx_stroke,pmhx_chf,pmhx_dementia,lab_sodium,lab_leukocyte,lab_mean_platelet_volume,lab_neutrophil,lab_inr,lab_mch,lab_creatinine,lab_mcv,lab_aptt,lab_platelet,lab_lymphocyte_percentage,lab_glucose,lab_neutrophil_percentage,lab_ldh,lab_prothrombin_activity,lab_urea,lab_lymphocyte,lab_crp,lab_rdw,lab_hemoglobin,lab_rbc,lab_hct,lab_potassium,lab_alt,lab_ast
count,200.000000,200.000000,0.0,200.000000,200.000000,200.000000,200.000000,200.000000,197.000000,200.000000,69.0,68.0,107.0,33.0,24.0,19.0,24.0,8.0,1.0,7.0,33.0,27.0,192.000000,192.000000,200.000000,192.000000,162.000000,200.000000,192.000000,200.000000,160.000000,192.000000,192.000000,191.000000,192.000000,177.000000,137.000000,191.000000,192.000000,164.000000,200.000000,192.000000,200.000000,200.000000,192.000000,190.000000,192.000000
mean,123.795000,64.535000,NaN,0.300000,38.018889,129.210000,73.920000,104.305000,86.842640,0.415000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,136.463542,9.831354,8.511500,8.288021,1.195679,28.929500,1.845833,87.729500,33.335000,239.187500,12.681291,159.486911,81.706100,1324.548023,13.807299,28.973822,1.290625,161.275427,14.580000,14.143229,4.642950,40.654500,4.664583,65.836842,100.317708
std,70.489266,16.925161,NaN,0.459408,0.906297,24.163511,14.333908,17.882418,12.152123,0.493958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,6.796883,8.623280,0.971409,8.728122,0.188933,2.127938,3.733601,5.378104,5.348346,92.035426,10.001863,85.592662,30.807665,1764.638309,2.317188,26.441681,5.031005,115.649612,1.827209,10.403169,0.662401,5.806553,6.657696,148.530188,249.939298
min,2.000000,16.000000,NaN,0.000000,36.277778,67.000000,29.000000,63.000000,20.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,111.000000,1.000000,6.700000,0.500000,0.800000,20.900000,0.300000,68.600000,14.200000,52.000000,0.814111,40.000000,14.047619,319.000000,9.600000,4.000000,0.100000,5.000000,12.400000,7.800000,2.830000,24.200000,2.700000,10.000000,19.000000
25%,63.750000,52.000000,NaN,0.000000,37.277778,114.750000,65.000000,91.000000,85.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,133.000000,5.900000,7.800000,4.400000,1.100000,28.000000,0.700000,85.000000,30.100000,182.000000,6.514681,110.000000,74.519152,815.000000,12.600000,12.500000,0.600000,64.650000,13.600000,12.300000,4.237500,36.975000,3.800000,25.000000,46.750000
50%,124.500000,65.000000,NaN,0.000000,37.944444,127.000000,73.000000,103.500000,90.000000,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,136.000000,7.850000,8.400000,6.400000,1.200000,29.200000,0.900000,88.350000,32.600000,220.000000,9.789205,129.000000,82.462366,1011.000000,13.400000,18.000000,0.800000,160.450000,14.100000,13.350000,4.655000,40.600000,4.100000,37.500000,66.000000
75%,183.250000,77.000000,NaN,1.000000,38.833333,142.250000,81.250000,113.250000,94.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,138.000000,11.400000,9.125000,9.750000,1.300000,30.300000,1.500000,91.225000,36.550000,289.250000,16.549546,177.000000,88.317726,1453.000000,14.400000,32.500000,1.200000,237.325000,15.100000,14.625000,5.080000,44.500000,4.500000,62.000000,98.000000
max,243.000000,97.000000,NaN,1.000000,40.000000,205.000000,131.000000,169.000000,100.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,164.000000,84.000000,11.400000,87.500000,2.200000,33.900000,41.400000,99.500000,60.200000,605.000000,83.214286,521.000000,455.729167,23097.000000,24.800000,147.000000,69.900000,535.500000,24.900000,155.000000,7.370000,60.900000,96.000000,1929.000000,3401.000000


In [13]:
# since in the end we are not running several models, but only one with the hm_hospitales variables
# we are filling with 0 variables that containg 100% missing values

hoboken_structured_data.ed_diagnosis=0

In [14]:
# we need the same columns available in this dataset that we have in the hm hospitales dataset
# so we are going to fill the missing ones with 0
hoboken_structured_data['lab_ddimer']=0

hoboken_structured_data=hoboken_structured_data[['patient ID','age','hospital_outcome','sex'
,'vitals_temp_ed_first','vitals_sbp_ed_first','vitals_dbp_ed_first','vitals_hr_ed_first','vitals_spo2_ed_first'
,'pmhx_diabetes','pmhx_hld','pmhx_htn','pmhx_ihd','pmhx_ckd','pmhx_copd','pmhx_asthma'
,'pmhx_activecancer','pmhx_chronicliver','pmhx_stroke','pmhx_chf','pmhx_dementia'
,'lab_sodium', 'lab_leukocyte', 'lab_mean_platelet_volume',
'lab_neutrophil', 'lab_inr', 'lab_mch', 'lab_creatinine', 'lab_mcv',
'lab_aptt', 'lab_platelet', 'lab_lymphocyte_percentage', 'lab_glucose',
'lab_neutrophil_percentage', 'lab_ldh', 'lab_prothrombin_activity',
'lab_urea', 'lab_lymphocyte', 'lab_crp', 'lab_rdw', 'lab_hemoglobin',
'lab_rbc', 'lab_hct', 'lab_potassium', 'lab_alt', 'lab_ast',
'lab_ddimer']]

categorical = ['sex','hospital_outcome', 'pmhx_diabetes', 'pmhx_hld', 'pmhx_htn', 'pmhx_ihd',
       'pmhx_ckd', 'pmhx_copd', 'pmhx_asthma', 'pmhx_activecancer',
       'pmhx_chronicliver', 'pmhx_stroke', 'pmhx_chf', 'pmhx_dementia']
print(TableOne(hoboken_structured_data,columns=hoboken_structured_data.columns.tolist(),categorical=categorical,groupby='hospital_outcome',pval=True))

                                            Grouped by hospital_outcome                                                         
                                                                Missing          Overall            0.0              1.0 P-Value
n                                                                                    200            117               83        
patient ID, mean (SD)                                                 0     123.8 (70.5)   106.8 (73.7)     147.7 (58.2)  <0.001
age, mean (SD)                                                        0      64.5 (16.9)    60.7 (16.1)      70.0 (16.7)  <0.001
hospital_outcome, n (%)              0.0                              0       117 (58.5)    117 (100.0)                   <0.001
                                     1.0                                       83 (41.5)                      83 (100.0)        
sex, n (%)                           FEMALE                           0        84 (42.0)      55 

# Data standardization

## Imputation and scale

In [ ]:
def impute_external(col_name,imputer_list,data):
    try:
        for name,imputer in imputer_list:
            if name == col_name:
                imputed_data = imputer.transform(data.to_numpy().reshape(-1, 1))
                imputed_data = imputed_data.ravel()
    except BaseException as e:
        print(f'Column: {col_name} encountered exception {e}')
    return pd.Series(imputed_data,name=col_name)

def scale_external(col_name,scaler_list,data):
    try:
        for name,scaler in scaler_list:
            if name == col_name:
                scaled_data = scaler.transform(data.to_numpy().reshape(-1, 1))
                scaled_data = scaled_data.ravel()
    except BaseException as e:
        print(f'Column: {col_name} encountered exception {e}')
    return pd.Series(scaled_data,name=col_name)

In [ ]:
#in the case of comorbidities, empty means 0.

hoboken_structured_data[['pmhx_diabetes', 'pmhx_hld', 'pmhx_htn', 'pmhx_ihd',
'pmhx_ckd', 'pmhx_copd', 'pmhx_asthma', 'pmhx_activecancer',
'pmhx_chronicliver', 'pmhx_stroke', 'pmhx_chf', 'pmhx_dementia']] = hoboken_structured_data[['pmhx_diabetes', 'pmhx_hld', 'pmhx_htn', 'pmhx_ihd',
'pmhx_ckd', 'pmhx_copd', 'pmhx_asthma', 'pmhx_activecancer',
'pmhx_chronicliver', 'pmhx_stroke', 'pmhx_chf', 'pmhx_dementia']].fillna(0)

In [ ]:
hoboken_structured_data_ = hoboken_structured_data.reset_index()
#hoboken_structured_data = hoboken_structured_data.drop(columns='admission_datetime')

#dropping all drug columns, there are zero patients with 'vasopressor' and 'intubation' - this needs to be verified with data owners
hoboken_structured_data_ = hoboken_structured_data_[[name for name in hoboken_structured_data_.columns if 'drug' not in name]]

def impute(series,method=None,missing_indicator=False): 
    """
    Wrapper function for sklearn's SimpleImputer

    Parameters
    ----------
    series: pd.Series
        a pd.Series to impute
    method: string
        passed to SimpleImputer's strategy parameter
    missing_indicator: bool
        logical flag to indicate if a missing value indicator column should be added

    Returns
    -------
    output_df: pd.DataFrame
        a pd.DataFrame containing the imputed series + missing indicator column
    """
    name = series.name
    si = SimpleImputer(strategy=method,add_indicator=missing_indicator)
    array = si.fit_transform(series.values.reshape(-1, 1))
    if missing_indicator:
        output_df = pd.DataFrame(array,columns=[name,name+'_missing'])
    else:
        output_df = pd.DataFrame(array,columns=[name])
    return output_df, si

def encode(series,drop_first=True):
    """
    Onehot encodes a categorical dataframe, dropping the first column

    Parameters
    ----------
    series: pd.Series
        a categorical pandas series
    drop_first: bool
        logical flag for whether the first category should be dropped

    Returns
    -------
    onehot_df: pd.DataFrame
        a pd.DataFrame containing the onehot encoded columns
    """
    onehot_df = pd.get_dummies(series,drop_first=drop_first)
    return onehot_df

def scale_center(series):
    """
    Wrapper function to scale and center a pd.Series using sklearn's Robust Scaler
    """
    rs = RobustScaler()
    array = rs.fit_transform(series.values.reshape(-1,1))
    array = array.flatten()
    series = pd.Series(array)
    return series, rs

any_missing = hoboken_structured_data_.columns[hoboken_structured_data_.isnull().sum() > 0].tolist()

imputer_list = []
scaler_list = []

for column in hoboken_structured_data_.columns.tolist():
    #Categorical features
    if ('pmhx' in column) or ('drug' in column) or (column in ['ed_diagnosis','sex']):
        if column in any_missing:
            #Mode imputation
            imputed,imputer = impute(hoboken_structured_data_[column],method='most_frequent')
            imputer_list += [(column,imputer)]
            hoboken_structured_data_ = hoboken_structured_data_.drop(columns=column).merge(imputed, left_index=True, right_index=True)
        if (len(hoboken_structured_data_[column].value_counts()) > 2) or (hoboken_structured_data_[column].dtype=='O'):
            #One hot encoding
            onehot_df = encode(hoboken_structured_data_[column])
            hoboken_structured_data_ = hoboken_structured_data_.drop(columns=column).merge(onehot_df, left_index=True, right_index=True)
    #Numeric features - vital signs, laboratory values
    elif ('age' in column) or ('vitals' in column) or ('lab' in column):
        if column in any_missing:
            #Median imputation
            imputed,imputer = impute(hoboken_structured_data_[column],method='median')
            imputer_list += [(column,imputer)]
            hoboken_structured_data_ = hoboken_structured_data_.drop(columns=column).merge(imputed, left_index=True, right_index=True)
        if 'spo2' not in column:
            #Winsorize to 1st and 99th percentile - excluding SpO2 which can normally take a value of 100%
            hoboken_structured_data_[column] = winsorize(hoboken_structured_data_[column],limits=(0.01,0.01))
        #Scale and center numeric columns
        hoboken_structured_data_[column],scaler = scale_center(hoboken_structured_data_[column])
        scaler_list += [(column,scaler)]
    
hoboken_structured_data_ = hoboken_structured_data_.set_index('patient ID')

# Final Table 1

In [ ]:
categorical = ['hospital_outcome', 'MALE',
'pmhx_chronicliver', 'pmhx_diabetes', 'pmhx_hld', 'pmhx_htn', 'pmhx_ihd', 'pmhx_ckd', 'pmhx_copd', 'pmhx_asthma', 'pmhx_activecancer', 'pmhx_stroke', 'pmhx_chf', 'pmhx_dementia',]

print(TableOne(hoboken_structured_data_,columns=hoboken_structured_data_.columns.tolist(),categorical=categorical,groupby='hospital_outcome',pval=True))

                                         Grouped by hospital_outcome                                                  
                                                             Missing       Overall           0.0           1.0 P-Value
n                                                                              242           138           104        
index, mean (SD)                                                   0  120.5 (70.0)  101.9 (71.9)  145.2 (59.3)  <0.001
age, mean (SD)                                                     0    -0.0 (0.6)    -0.2 (0.6)     0.1 (0.6)  <0.001
hospital_outcome, n (%)              0.0                           0    138 (57.0)   138 (100.0)                <0.001
                                     1.0                                104 (43.0)                 104 (100.0)        
vitals_temp_ed_first, mean (SD)                                    0     0.1 (0.6)     0.0 (0.6)     0.1 (0.6)   0.727
vitals_sbp_ed_first, mean (SD)                  

# Append Image features

## Features ingestion

In [ ]:
hoboken_image_feature = pd.read_csv('features/test-hoboken_image_feature.csv', header=None)
hoboken_image_names = pd.read_csv('features/test-hoboken_image_names.csv', header=None)

## Appending files

In [ ]:
hoboken_image_feature = pd.concat([hoboken_image_feature.reset_index(drop=True), hoboken_image_names], axis=1)

## Naming all features

In [ ]:
hoboken_image_feature_names = ['feat_' + s for s in [str(i).zfill(4) for i in range(1,65)] ]
hoboken_image_feature_names = np.append(hoboken_image_feature_names,['path'])
hoboken_image_feature.columns = hoboken_image_feature_names

## Extracting patient id from feature dataframe

In [ ]:
hoboken_image_feature['patient ID']=hoboken_image_feature.path.str.extract('(\d+)').astype('int32')

invalid escape sequence \d
invalid escape sequence \d


In [ ]:
print('Number of patients with CXR available')
hoboken_image_feature['patient ID'].nunique()

Number of patients with CXR available


201

# Final Join

In [ ]:
hoboken_image_feature_data= hoboken_structured_data_.merge(hoboken_image_feature, how='inner', on='patient ID')
print('Final number of patients in the combined dataset (containing both images and structured data)')
print(hoboken_image_feature_data['patient ID'].nunique())
# dropping index column to avoid confusions
del hoboken_image_feature_data['index']

Final number of patients in the combined dataset (containing both images and structured data)
201


# Columns renaming and final export



In [ ]:
hoboken_image_feature_data=hoboken_image_feature_data.rename(columns={"patient ID": "PATIENT ID"})
hoboken_image_feature_data=hoboken_image_feature_data.rename(columns={"hospital_outcome": "expired_30_days"})

hoboken_image_feature_data.to_csv('final_validation_datasets/hoboken_image_feature_data.csv',index=False)